<a href="https://colab.research.google.com/github/VietDangiT/ETL_Pipeline/blob/main/Data%20Engineer%20for%20ETL%3A%20Data%20Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingest Data using dlt and Python

In [2]:
!pip install dlt duckdb pandas requests


Fetch Data from CoinGecko API

In [ ]:
import requests

url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1
}
response = requests.get(url, params=params)
data = response.json()
data[:1]  # xem thử 1 item



Định nghĩa resource với dlt

In [9]:
import dlt

@dlt.resource
def coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {"vs_currency": "usd", "order": "market_cap_desc", "per_page": 10, "page": 1}
    yield requests.get(url, params=params).json()


Tạo pipeline và ghi dữ liệu vào DuckDB

In [10]:
pipeline = dlt.pipeline(
    pipeline_name="crypto_pipeline",
    destination="duckdb",
    dataset_name="crypto_data"
)

load_info = pipeline.run(coins())
print(load_info)


2025-07-03 15:26:12,101|[WARNING]|385|131990784954368|dlt|validate.py|verify_normalized_table:57|In schema `crypto`: The following columns in table 'coins' did not receive any data during this load and therefore could not have their types inferred:
  - roi

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'roi': {'data_type': 'text'}})



Pipeline crypto_pipeline load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset crypto_data
The duckdb destination used duckdb:////content/crypto_pipeline.duckdb location to store data
Load package 1751556371.9137156 is LOADED and contains no failed jobs


Kiểm tra dữ liệu đã ghi vào DuckDB

In [11]:
import duckdb

con = duckdb.connect("dlt_pipeline.duckdb")
con.sql("SELECT * FROM crypto_data.coins LIMIT 5").df()


InvalidInputException: Invalid Input Error: Python Object "coins" of type "DltResource" found on line "/tmp/ipython-input-11-2369424038.py:4" not suitable for replacement scans.
Make sure that "coins" is either a pandas.DataFrame, duckdb.DuckDBPyRelation, pyarrow Table, Dataset, RecordBatchReader, Scanner, or NumPy ndarrays with supported format

Extracting data with dlt: dlt RestAPI Client

In [ ]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import JSONLinkPaginator

client = RESTClient(
    base_url="https://api.example.com",
    auth=BearerTokenAuth(token="YOUR_TOKEN"),
    paginator=JSONLinkPaginator(next_url_path="pagination.next"),
    data_selector="data"
)

for page in client.paginate("/endpoint"):
    handle(page)  # page là dữ liệu JSON đã chuẩn hóa
